In [1]:
if (FALSE) {
    library(devtools)
    library(httr)
    set_config(config(ssl_verifypeer = 0L))
    options(repos=structure(c(CRAN="http://cran.rstudio.com/")))
    install.packages(c("phytools","amap","RColorBrewer","colorspace","dendextend","NMF","MASS","caper","pvclust"))
}
library(phytools)
library(amap)
library(RColorBrewer)
library(colorspace)
library(dendextend)
library(NMF)
library(MASS)
library(caper)
library(pvclust)
library(Rtsne)
options(stringsAsFactors = FALSE)

Loading required package: ape
Loading required package: maps

---------------------
Welcome to dendextend version 1.7.0
Type citation('dendextend') for how to cite the package.

Type browseVignettes(package = 'dendextend') for the package vignette.
The github page is: https://github.com/talgalili/dendextend/

Suggestions and bug-reports can be submitted at: https://github.com/talgalili/dendextend/issues
Or contact: <tal.galili@gmail.com>

	To suppress this message use:  suppressPackageStartupMessages(library(dendextend))
---------------------


Attaching package: ‘dendextend’

The following object is masked from ‘package:phytools’:

    untangle

The following objects are masked from ‘package:ape’:

    ladderize, rotate

The following object is masked from ‘package:stats’:

    cutree

Loading required package: pkgmaker
Loading required package: registry

Attaching package: ‘pkgmaker’

The following object is masked from ‘package:base’:

    isNamespaceLoaded

Loading required package

In [17]:
# Parameter
date_sra = '2018_5_1'
date_orthofinder = 'Mar22'
selected_tissues = c('brain','heart','kidney','liver','ovary','testis')
dir_ensembl = '/Users/kf/Dropbox/kfdata/02_Data/my_db/Ensembl/release-91/'
dir_curated_transcriptome = paste0(dir_ensembl, 'curated_transcriptome/')
dir_tc = paste0(dir_curated_transcriptome, date_sra, '/', 'tc/')
dir_uncorrected_tissue_mean = paste0(dir_curated_transcriptome, date_sra, '/', 'uncorrected_tissue_mean/')
dir_tissue_mean = paste0(dir_curated_transcriptome, date_sra, '/', 'tissue_mean/')
dir_sra = paste0(dir_curated_transcriptome, date_sra, '/', 'sra/')
file_outsra = paste0(dir_ensembl, 'sra/', date_sra, '/sra_table_amalgamated_', date_sra, '.tsv')
file_species_tree = '/Users/kf/Dropbox/kfdata/02_Data/my_db/Ensembl/release-91/timetree/species_timetree.nwk'

sptree = ladderize(read.newick(file_species_tree), right=TRUE)
sp_orders = rev(sptree$tip.label)
label_orders = c()
for (tissue in selected_tissues) {
    label_orders = c(label_orders, paste(sp_orders, tissue, sep='_'))
}

dir_orthofinder = paste0(dir_ensembl, 'orthofinder/')
file_singlecopy = paste0(dir_orthofinder, 'Results_', date_orthofinder, '/SingleCopyOrthogroups.txt')
file_orthogroup = paste0(dir_orthofinder, 'Results_', date_orthofinder, '/Orthogroups.csv')
single_orthogroups = read.table(file_singlecopy, header=FALSE)$V1
df_og = read.table(file_orthogroup, header=TRUE, sep='\t', row.names=1)
df_singleog = df_og[(rownames(df_og) %in% single_orthogroups),]
for (col in 1:ncol(df_singleog)) {
    df_singleog[,col] = sub('.*_', '', df_singleog[,col])
}
df_og = NULL
spp_filled = colnames(df_singleog)
spp = sub('_', ' ', spp_filled)
cat('num selected species:', length(spp), '\n')
cat('selected species:', spp, '\n')

num selected species: 21 
selected species: Anolis carolinensis Astyanax mexicanus Bos taurus Callithrix jacchus Canis lupus Chinchilla lanigera Danio rerio Gadus morhua Gallus gallus Homo sapiens Macaca mulatta Monodelphis domestica Mus musculus Oreochromis niloticus Ornithorhynchus anatinus Oryctolagus cuniculus Oryzias latipes Ovis aries Rattus norvegicus Sus scrofa Xenopus tropicalis 


In [3]:
add_color_to_sra = function(df, selected_tissues) {
    df = df[,(!colnames(df) %in% c('bp_color','sp_color','tissue_color'))]
    scientific_name = as.character(df$scientific_name)
    tissue = as.character(df$tissue)
    scientific_name_unique = scientific_name[!duplicated(scientific_name)]
    tissue_unique = tissue[!duplicated(tissue)]
    if (length(selected_tissues) <= 8) {
        tissue_color = brewer.pal(length(unique(tissue)), "Dark2")
        sp_color = rainbow_hcl(length(unique(scientific_name)), c=100)
    } else if (length(selected_tissues) <= 12) {
        tissue_color = brewer.pal(length(unique(tissue)), "Paired")
        sp_color = rainbow_hcl(length(unique(scientific_name)), c=100)
    } else {
        tissue_color = rainbow_hcl(length(selected_tissues), c=100)
        sp_color = rainbow_hcl(length(unique(scientific_name)), c=150)
    }
    df_tissue = data.frame(tissue=tissue_unique, tissue_color=tissue_color[1:length(tissue_unique)], stringsAsFactors=FALSE)
    df_sp = data.frame(scientific_name=scientific_name_unique, sp_color=sp_color[1:length(scientific_name_unique)], stringsAsFactors=FALSE)
    df = merge(df, df_sp, sort=FALSE, all.y=FALSE)
    df = merge(df, df_tissue, sort=FALSE, all.y=FALSE)
    if ('bioproject' %in% colnames(df)) {
        bioproject = as.character(df$bioproject)
        if (length(selected_tissues) <= 8) {
            bp_color = rainbow_hcl(length(unique(bioproject)), c=50)
        } else if (length(selected_tissues) <= 12) {
            bp_color = rainbow_hcl(length(unique(bioproject)), c=50)
        } else {
            bp_color = rainbow_hcl(length(unique(bioproject)), c=50)
        }
        df_bp = data.frame(bioproject=unique(bioproject), bp_color=bp_color[1:length(unique(bioproject))], stringsAsFactors=FALSE)
        df = merge(df, df_bp, sort=FALSE, all.y=FALSE)
    }
    return(df)
}

sort_labels = function(df_label, label_orders) {
    df_tmp = data.frame()
    for (lo in label_orders) {
        splits = strsplit(lo, '_')[[1]]
        scientific_name = paste(splits[1], splits[2])
        tissue = splits[3]
        df_tmp = rbind(df_tmp, df_label[(df_label$scientific_name==scientific_name)&(df_label$tissue==tissue),])
    }
    return(df_tmp)
}

sort_averaged_tc = function(tc) {
    split_colnames = strsplit(colnames(tc), "_")
    genus_names = c()
    specific_names = c()
    tissue_names = c()
    for (i in 1:length(split_colnames)) {
        genus_names = c(genus_names, split_colnames[[i]][1])
        specific_names = c(specific_names, split_colnames[[i]][2])
        tissue_names = c(tissue_names, split_colnames[[i]][3])
    }
    colname_order = order(tissue_names, genus_names, specific_names)
    tc = tc[, colname_order]
    return(tc)
}

color_children2parent = function(node) {
    if (length(node)==2) {
        child1_color = attributes(node[[1]])$edgePar[['col']]
        child2_color = attributes(node[[2]])$edgePar[['col']]
        if ((!is.null(child1_color))&(!is.null(child2_color))) {
            if (child1_color==child2_color) {
                attributes(node)$edgePar[['col']] = child1_color
            }
        }
    }
    return(node)
}

map_color = function(redundant_variables, c) {
    uniq_var = unique(redundant_variables)
    uniq_col = rainbow_hcl(length(uniq_var), c=c)
    df_unique = data.frame(var=uniq_var, col=uniq_col, stringsAsFactors=FALSE)
    df_redundant = data.frame(var=redundant_variables, order=seq(1, length(redundant_variables)), stringsAsFactors=FALSE)
    df_redundant = merge(df_redundant, df_unique, by="var", all.x=TRUE, stringsAsFactors=FALSE)
    df_redundant = df_redundant[order(df_redundant$order),]
    return(df_redundant$col)
}

draw_multisp_heatmap = function(tc, df_label) {
    tc_dist_matrix = cor(tc, method='pearson')
    tc_dist_matrix[is.na(tc_dist_matrix)] = 0
    ann_label = df_label[,c('scientific_name','tissue')]
    colnames(ann_label) = c('species', 'organ')
    sp_color = df_label$sp_color[!duplicated(df_label$sp_color)]
    sp_color = sp_color[order(df_label$scientific_name[!duplicated(df_label$scientific_name)])]
    organ_color = df_label$tissue_color[!duplicated(df_label$tissue_color)]
    organ_color = organ_color[order(df_label$tissue[!duplicated(df_label$tissue)])]
    ann_color = list(species=sp_color, organ=organ_color)
    breaks = c(0, seq(0.3, 1, 0.01))
    aheatmap(tc_dist_matrix, color="-RdYlBu2:71", Rowv=NA, Colv=NA, revC=TRUE, legend=TRUE, breaks=breaks, 
                       annCol=ann_label, annRow=ann_label, annColors=ann_color, annLegend=FALSE, labRow=NA, labCol=NA)
}

draw_multisp_dendrogram = function(tc, df_label, sra, nboot, cex.xlab, cex.yaxis, pvclust_file='pvclust.RData') {
    colnames(tc) = sub("_.*","",sub('_',' ',colnames(tc)))
    dist_fun = function(x){Dist(t(x), method='pearson')}
    if (file.exists(pvclust_file)) {
        if (file.info(pvclust_file)$size) {
            print('pvclust file found.')
            load(pvclust_file)
        }
    } else {
        print('no pvclust file found. Start bootstrapping.')
        result = pvclust(tc, method.dist=dist_fun, method.hclust="average", nboot=nboot, parallel=FALSE) # UPGMA
        save(result, file=pvclust_file)
    }
    dend = as.dendrogram(result)
    dend_colors = df_label$tissue_color[order.dendrogram(dend)]
    label_colors = df_label$sp_color[order.dendrogram(dend)]
    labels_colors(dend) = label_colors
    dend_labels <- sra$run[order.dendrogram(dend)]
    dend <- color_branches(dend, labels=dend_labels, col=dend_colors)
    dend <- set(dend, "branches_lwd", 2)
    for (i in 1:ncol(tc)) {
        dend = dendrapply(dend, color_children2parent)
    }
    par(cex=cex.xlab)
    plot(dend, las=1, yaxt='n')
    text(result, print.num=FALSE, cex=1, col.pv='black')
    par(cex=cex.yaxis)
    axis(2, las=1)
    mtext(text='Distance', side=2, line=4, cex=cex.yaxis)
    
    n = ncol(tc)
    f = 100
    tissue_unique = unique(sra$tissue)
    sp_unique = unique(sra$scientific_name)
    bp_unique = unique(sra$bioproject)
    tissue_color_unique = unique(sra$tissue_color)
    sp_color_unique = unique(sra$sp_color)
    bp_color_unique = unique(sra$bp_color)
    legend_text = c(as.character(tissue_unique), "", as.character(sp_unique), "", as.character(bp_unique))
    legend_bg = c(tissue_color_unique, "white", sp_color_unique, "white", bp_color_unique)
    legend_fg = c(rep("black", length(tissue_color_unique)), "white", rep("black", length(sp_color_unique)), "white", rep("black", length(bp_color_unique)))
    #plot.new() ; par(mar=c(0,0,0,0))
    #legend("center", legend=legend_text, cex=1, pch=22, lty=0, lwd=1, pt.bg=legend_bg, col=legend_fg)
}

draw_multisp_pca = function(tc, df_label) {
    tc_dist_matrix = cor(tc, method='pearson')
    tc_dist_matrix[is.na(tc_dist_matrix)] = 0
    set.seed(1)
    pca = prcomp(tc_dist_matrix)
    xlabel = paste0("PC 1 (", round(summary(pca)$importance[2,1]*100, digits=1), "%)")
    ylabel = paste0("PC 2 (", round(summary(pca)$importance[2,2]*100, digits=1), "%)")
    plot(pca$rotation[,1], pca$rotation[,2], pch=21, cex=2, lwd=1, bg=df_label$tissue_color, col=df_label$sp_color, xlab=xlabel, ylab=ylabel, las=1)
}


draw_multisp_mds = function(tc, df_label) {
    tc_dist_dist = Dist(t(tc), method='pearson') + 0.000000001
    tc_dist_dist[is.na(tc_dist_dist)] = 1
    set.seed(1)
    try_out = tryCatch(
        {isoMDS(tc_dist_dist, k=2, maxit=100)},
        error = function(a){return("MDS failed.")}
    )
    if (mode(try_out)=="character") {
        cat('MDS failed.\n')
        plot(c(0, 1), c(0, 1), ann = F, bty = 'n', type = 'n', xaxt = 'n', yaxt = 'n')
    } else {
        mds <- try_out
        plot(mds$points[,1], mds$points[,2], pch=21, cex=2, lwd=1, bg=df_label$tissue_color, col=df_label$sp_color, xlab="MDS dimension 1", ylab="MDS dimension 2", las=1)
    }
}

draw_multisp_tsne = function(tc, df_label) {
    perplexity = min(30, floor(ncol(tc)/4))
    set.seed(1)
    out_tsne = Rtsne(as.matrix(t(tc)), theta=0, check_duplicates=FALSE, verbose=FALSE, perplexity=perplexity, dims=2)
    try_out = tryCatch(
        {
            plot(out_tsne$Y[,1], out_tsne$Y[,2], pch=21, cex=2, lwd=1, bg=df_label$tissue_color, col=df_label$sp_color, 
                 xlab="t-SNE dimension 1", ylab="t-SNE dimension 2", las=1)
        },
        error = function(a){return("t-SNE plot failed.")}
    )
    if (mode(try_out)=="character") {
        cat('t-SNE failed.\n')
        plot(c(0, 1), c(0, 1), ann = F, bty = 'n', type = 'n', xaxt = 'n', yaxt = 'n')
    }    
  
}

draw_multisp_legend = function(df_label) {
    tissue_unique = df_label$tissue[!duplicated(df_label$tissue)]
    sp_unique = df_label$scientific_name[!duplicated(df_label$scientific_name)]
    tissue_color_unique = df_label$tissue_color[!duplicated(df_label$tissue_color)]
    sp_color_unique = df_label$sp_color[!duplicated(df_label$sp_color)]
    toumei=rgb(1,1,1,0)
    legend_text = c('Tissue', as.character(tissue_unique), "", 'Species', as.character(sp_unique))
    legend_bg = c(toumei, tissue_color_unique, toumei, toumei, rep(toumei, length(sp_color_unique)))
    legend_fg = c(toumei, rep(toumei, length(tissue_color_unique)), toumei, toumei, sp_color_unique)
    legend_pch = c(1, rep(21,length(tissue_color_unique)), 1, 1, rep(1,length(sp_color_unique)))
    legend_font = c(2, rep(1, length(tissue_color_unique)), 1, 2, rep(3, length(sp_color_unique)))
    plot.new()
    legend("right", legend=legend_text, pt.cex=1, pch=legend_pch, lty=0, lwd=2, pt.bg=legend_bg, col=legend_fg, cex=cex_axis, text.font=legend_font)

}

In [4]:
files = list.files(dir_sra)
df_sra = data.frame()
for (file in files) {
    sra_path = paste0(dir_sra, file)
    tmp_sra = read.table(sra_path, header=TRUE, sep='\t', quote='', comment.char='')
    df_sra = rbind(df_sra, tmp_sra)
}
write.table(df_sra, file_outsra, row.names=FALSE, sep='\t')

df_sra = df_sra[(df_sra$tissue %in% selected_tissues)&(df_sra$scientific_name %in% spp),]

In [5]:
tcs = list()
tcs[['uncorrected']] = list()
tcs[['corrected']] = list()
uncorrected_files = list.files(dir_uncorrected_tissue_mean)
corrected_files = list.files(dir_tissue_mean)
for (sp in spp_filled) {
    uncorrected_file = uncorrected_files[startsWith(uncorrected_files, sp)]
    corrected_file = corrected_files[startsWith(corrected_files, sp)]
    uncorrected_path = paste0(dir_uncorrected_tissue_mean, uncorrected_file)
    corrected_path = paste0(dir_tissue_mean, corrected_file)
    tcs[['uncorrected']][[sp]] = read.table(uncorrected_path, header=TRUE, row.names=1, sep='\t')
    tcs[['corrected']][[sp]] = read.table(corrected_path, header=TRUE, row.names=1, sep='\t')
}
cat('done!\n')

done!


In [15]:
# Single-copy ortholog extraction

ortholog = list()
ortholog[['uncorrected']] = df_singleog
ortholog[['corrected']] = df_singleog
row_names = rownames(ortholog[['corrected']])
selected_species = spp
for (d in c('uncorrected', 'corrected')) {
    cat(d, '\n')
    for (sp in spp) {
        sp_filled = sub(" ", "_", sp)
        tc = tcs[[d]][[sp_filled]]
        colnames(tc) = paste(sp_filled, colnames(tc), sep='_')
        ortholog[[d]] = merge(ortholog[[d]], tc, by.x=sp_filled, by.y="row.names", all.x=TRUE, all.y=FALSE, sort=FALSE)
    }
    num_remove_col = length(selected_species)
    ortholog[[d]] = ortholog[[d]][,-(1:num_remove_col)]
    rownames(ortholog[[d]]) = row_names
    ortholog[[d]] = sort_averaged_tc(ortholog[[d]])
    #ortholog[[d]][ortholog[[d]]<0] = 0
    ortholog[[d]] = ortholog[[d]][, label_orders]
}
cat(nrow(ortholog[[d]]), 'orthologs detected.\n')

uncorrected 
corrected 
1377 orthologs detected.


In [35]:
df_labels = list()
for (tpm in c('uncorrected','corrected')) {
    if (tpm=='uncorrected') {
        sra_tmp = df_sra
    } else if (tpm=='corrected') {
        sra_tmp = df_sra[(df_sra$exclusion=='no'),]
    }
    
    df_label = unique(sra_tmp[,c('scientific_name','tissue')])
    categories = list(scientific_name=sra_tmp$scientific_name, tissue=sra_tmp$tissue)

    df_bp = aggregate(sra_tmp$bioproject, by=categories, function(x){length(unique(x))})
    colnames(df_bp) = c('scientific_name','tissue','num_bp')
    df_label = merge(df_label, df_bp, all.x=TRUE, all.y=FALSE)

    df_exp = aggregate(sra_tmp$experiment, by=categories, function(x){length(unique(x))})
    colnames(df_exp) = c('scientific_name','tissue','num_exp')
    df_label = merge(df_label, df_exp, all.x=TRUE, all.y=FALSE)
    df_label = df_label[order(df_label$tissue, df_label$scientific_name),]
    df_label = sort_labels(df_label, label_orders)
    df_label = add_color_to_sra(df_label, selected_tissues)
    df_label = sort_labels(df_label, label_orders)
    df_labels[[tpm]] = df_label
}

In [25]:
do_heatmap=TRUE
do_dendrogram=TRUE
do_pca_mds=TRUE

if (do_heatmap) {
    file_name='Multisp.SVA.heatmap.pdf'
    pdf(file_name, height=3.3, width=7.2) # full figure size = 9.7 x 7.2
    layout_matrix=matrix(c(
        1,2,2,2,2,2,2,2,2,2,
        1,3,3,3,3,3,3,3,3,3),
        2,10,byrow=TRUE)
    layout(t(layout_matrix))
    par(mar=c(0,0,0,0))
    plot(c(0, 1), c(0, 1), ann = F, bty = 'n', type = 'n', xaxt = 'n', yaxt = 'n')
    text(0.27,0.5,'Uncorrected', srt=0, cex=2)
    text(0.80,0.5,'Corrected', srt=0, cex=2)
    for (tpm in c('uncorrected','corrected')) {
        tc = ortholog[[tpm]]
        df_label = df_labels[[tpm]]
        par(mar=c(0,0,0,0))
        draw_multisp_heatmap(tc=tc, df_label=df_label)
    }
    graphics.off()
}

if (do_dendrogram) {
    file_name='Multisp.SVA.dendrogram.pdf'
    pdf(file_name, height=2.5, width=7.2) # full figure size = 9.7 x 7.2
    layout_matrix=matrix(c(1,2),2,1,byrow=TRUE)
    layout(t(layout_matrix))
    for (tpm in c('uncorrected','corrected')) {
        tc = ortholog[[tpm]]
        df_label = df_labels[[tpm]]
        par(cex=0.5, mar=c(10,5.5,0,0), mgp=c(4, 0.7, 0))
        pvclust_file=paste0('Multisp.pvclust.',tpm,'.RData')
        draw_multisp_dendrogram(tc=tc, df_label=df_label, sra=df_sra, pvclust_file=pvclust_file, 
                                nboot=1, cex.xlab=0.3, cex.yaxis=0.5)
    }
    graphics.off()
}

if (do_pca_mds) {
    par(cex=1)
    cex_axis=0.7
    cex_plot=1
    file_name='Multisp.SVA.PCA.MDS.pdf'
    pdf(file_name, height=4.5, width=7.2) # full figure size = 9.7 x 7.2
    layout_matrix = matrix(c(1,1,1,3,3,3,5,5,2,2,2,4,4,4,5,5),2,8,byrow=TRUE)
    layout(layout_matrix)
    for (tpm in c('uncorrected','corrected')) {
        tc = ortholog[[tpm]]
        df_label = df_labels[[tpm]]
        par(mar=c(4,4,0.1,1)); draw_multisp_pca(tc=tc, df_label=df_label)
        par(mar=c(4,4,0.1,1)); draw_multisp_tsne(tc=tc, df_label=df_label)
        #par(mar=c(4,4,0.1,1)); draw_multisp_mds(tc=tc, df_label=df_label)
    }
    par(mar=c(0,0,0,0)); draw_multisp_legend(df_label)
    graphics.off()
}


In [26]:
savefig=TRUE
do_combined=TRUE
panel_cex=1
chars = c('A','B','C','D','E','G','F','H')

if (do_combined) {
    if (savefig) {
        file_name='Multisp.SVA.combined.pdf'
        pdf(file_name, height=9.7, width=7.2) # full figure size = 9.7 x 7.2        
    }　else {
        options(repr.plot.height=9.7, repr.plot.width=7.2)
    }
    layout_matrix=matrix(c(
        1,1,1,1,2,2,2,2,
        3,3,3,3,4,4,4,4,
        3,3,3,3,4,4,4,4,
        3,3,3,3,4,4,4,4,
        3,3,3,3,4,4,4,4,
        3,3,3,3,4,4,4,4,
        3,3,3,3,4,4,4,4,
        3,3,3,3,4,4,4,4,
        3,3,3,3,4,4,4,4,
        5,5,5,5,6,6,6,6,
        5,5,5,5,6,6,6,6,
        5,5,5,5,6,6,6,6,
        5,5,5,5,6,6,6,6,
        5,5,5,5,6,6,6,6,
        5,5,5,5,6,6,6,6,
        7,7,7,9,9,9,11,11,
        7,7,7,9,9,9,11,11,
        7,7,7,9,9,9,11,11,
        7,7,7,9,9,9,11,11,
        7,7,7,9,9,9,11,11,
        7,7,7,9,9,9,11,11,
        8,8,8,10,10,10,11,11,
        8,8,8,10,10,10,11,11,
        8,8,8,10,10,10,11,11,
        8,8,8,10,10,10,11,11,
        8,8,8,10,10,10,11,11,
        8,8,8,10,10,10,11,11),
    27,8,byrow=TRUE)
    layout(layout_matrix)
    par(cex=1, xpd=T, mar=c(0,0,0,0)); plot(c(0, 1), c(0, 1), ann=F, bty='n',type='n',xaxt='n', yaxt='n')
    text(0.50,0.31,'Uncorrected', srt=0, cex=1, font=1, adj=c(0.5,1))
    text(-0.04, 0.31, chars[1], srt=0, font=2, cex=panel_cex, adj=c(0,1)) ; chars = chars[2:length(chars)]
    par(cex=1, xpd=T, mar=c(0,0,0,0)); plot(c(0, 1), c(0, 1), ann=F, bty='n',type='n',xaxt='n', yaxt='n')
    text(0.50,0.31,'Corrected', srt=0, cex=1, font=1, adj=c(0.5,1))
    text(-0.04, 0.31, chars[1], srt=0, font=2, cex=panel_cex, adj=c(0,1)) ; chars = chars[2:length(chars)]
    for (tpm in c('uncorrected','corrected')) {
        tc = ortholog[[tpm]]
        df_label = df_labels[[tpm]]
        par(mar=c(0,0,0,0))
        draw_multisp_heatmap(tc=tc, df_label=df_label)
        text(0.01, 0.93, 'Sp', srt=90, font=1, cex=0.7)
        text(0.05, 0.93, 'Tis', srt=90, font=1, cex=0.7)
        text(0.805, 0.97, 'Sp', srt=0, font=1, cex=0.7)
        text(0.805, 0.93, 'Tis', srt=0, font=1, cex=0.7)
    }
    for (tpm in c('uncorrected','corrected')) {
        tc = ortholog[[tpm]]
        df_label = df_labels[[tpm]]
        par(cex=0.5, mar=c(11,5.5,0,0), mgp=c(4, 0.7, 0))
        pvclust_file=paste0('Multisp.pvclust.',tpm,'.RData')
        draw_multisp_dendrogram(tc=tc, df_label=df_label, sra=df_sra, pvclust_file=pvclust_file, nboot=1000, cex.xlab=0.3, cex.yaxis=0.5)
        par(new=T, cex=1, xpd=T, mar=c(0,0,0,0)); plot(c(0, 1), c(0, 1), ann=F, bty='n',type='n',xaxt='n', yaxt='n')
        text(-0.04, 1.03, chars[1], srt=0, font=2, cex=panel_cex, adj=c(0,1)) ; chars = chars[2:length(chars)]
    }
    par(mar=c(0,0,0,0))
    for (tpm in c('uncorrected','corrected')) {
        tc = ortholog[[tpm]]
        df_label = df_labels[[tpm]]
        par(mar=c(4,4,0.1,1)); draw_multisp_pca(tc=tc, df_label=df_label)
        par(new=T, cex=1, xpd=T, mar=c(0,0,0,0)); plot(c(0, 1), c(0, 1), ann=F, bty='n',type='n',xaxt='n', yaxt='n')
        text(-0.04, 1.03, chars[1], srt=0, font=2, cex=panel_cex, adj=c(0,1)) ; chars = chars[2:length(chars)]
        par(mar=c(4,4,0.1,1)); draw_multisp_tsne(tc=tc, df_label=df_label)
        par(new=T, cex=1, xpd=T, mar=c(0,0,0,0)); plot(c(0, 1), c(0, 1), ann=F, bty='n',type='n',xaxt='n', yaxt='n')
        text(-0.04, 1.03, chars[1], srt=0, font=2, cex=panel_cex, adj=c(0,1)) ; chars = chars[2:length(chars)]
    }
    par(mar=c(0,0,0,0), xpd=T); draw_multisp_legend(df_label)
    if (savefig) {
        graphics.off()
    }
}

print('Completed!')

[1] "pvclust file found."
[1] "pvclust file found."
[1] "Completed!"


In [33]:
file_name='Multisp.SVA.t-SNE.pdf'
pdf(file_name, height=2, width=2.5) # full figure size = 9.7 x 7.2       
tpm = 'corrected'
tc = ortholog[[tpm]]
df_label = df_labels[[tpm]]
par(mar=c(2.5,2.5,0.3,0.1), cex=1, ps=8, mgp=c(1.5, 0.7, 0)); draw_multisp_tsne(tc=tc, df_label=df_label)
dev.off()

pdf 
  2